# My Book Recommender System (Beginner Friendly)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
# Load the datasets
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

In [ ]:
# Filter out zero ratings
ratings = ratings[ratings['Book-Rating'] > 0]

# Only include books rated by more than 50 users
popular_books = ratings['ISBN'].value_counts() > 50
ratings = ratings[ratings['ISBN'].isin(popular_books[popular_books].index)]

In [ ]:
# Create pivot table
pivot_table = ratings.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)

In [ ]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(pivot_table)

In [ ]:
# Map ISBNs to Book Titles
isbn_to_title = books.set_index('ISBN')['Book-Title'].to_dict()
pivot_table.index = pivot_table.index.map(isbn_to_title)

In [ ]:
# Prepare popular books dataframe
top_rated = ratings.groupby('ISBN').agg({'Book-Rating': ['count', 'mean']})
top_rated.columns = ['rating_count', 'avg_rating']
top_rated = top_rated.sort_values(by='rating_count', ascending=False).head(50)
book_info = books.drop_duplicates('ISBN').set_index('ISBN')
top_rated = top_rated.join(book_info, how='left')
top_rated = top_rated[['Book-Title', 'Book-Author', 'Image-URL-M', 'rating_count', 'avg_rating']]

In [ ]:
# Save required files
pickle.dump(top_rated, open('popular_books.pkl', 'wb'))
pickle.dump(pivot_table, open('pivot_table.pkl', 'wb'))
pickle.dump(books, open('book_info.pkl', 'wb'))
pickle.dump(similarity_matrix, open('similarity_matrix.pkl', 'wb'))

print("✅ All files saved. You can now run the Flask app.")